# Critical Scale Invariance in a Healthy Human Heart Rate

### Description

As described in the [paper](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.93.178103) (Physics Review Letters, V93 17, 2004), a robust scale-invariance is manifested in the probability density function (PDF) of detrended healthy human heart rate increments, which is preserved not only in a quiescent condition, but also in a dynamic state where the mean level of the heart rate is dramatically changing. This scale-independent and fractal structure is markedly different from the scale-dependent PDF evolution observed in a turbulentlike, cascade heart rate model. These results strongly support the view that a healthy human heart rate is controlled to converge continually to a critical state.

### Assignment

to be provided

### Contacts